# Simple synchronism test of the XCOM block

The general idea is that the boards are locked to a common reference, a "PPS" signal comming from a GPS for example. In this case we generate a false PPS in the HP 8110A generator. It has two synchronous channels and give us a "typical" PPS signal (200ms high, 800ms low). There is a "leader" board and a "follower" board. The boards are inter-connected through xcom blocks. 

### Hardware setup
We'll use two ZCU216 boards for this setup. The RF cabling is similar to what we use for the standard demos (DAC 228_0 to ADC 224_0), except that the second loopback goes between boards (DAC 229_2 of each board to ADC 224_1 of the other).

# First test the generators and acquisition
The idea here is to test the loopback connection like in 00_intro.ipynb

In [ ]:
# jupyter setup boilerplate
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numpy as np
from qick import *
# for now, all the tProc v2 classes needto be individually imported (can't use qick.*)
# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

In [ ]:
qick.get_version()

In [ ]:
soc = QickSoc('./tprocv2r21_standard.bit')
soccfg = soc
#print(soccfg)

In [ ]:
# ZCU216 standard FW, full-speed gen
GEN_CH = 0
RO_CH = 0
TRIG_TIME = 0.40
FREQ = 500

In [ ]:
class MultiPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        ramp_len = 0.2
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myflattop", ro_ch=ro_ch, 
                       style="flat_top", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       length=0.1,
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mygaus", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=0.2, 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0,
                      )

        self.add_pulse(ch=gen_ch, name="myflattop2", ro_ch=ro_ch, 
                       style="flat_top", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       length=0.1,
                       phase=90,
                       gain=1.0, 
                      )

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        self.pulse(ch=cfg['gen_ch'], name="myflattop", t=0)
        self.pulse(ch=cfg['gen_ch'], name="mygaus", t=0.4)
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0.8)
        self.pulse(ch=cfg['gen_ch'], name="myflattop2", t=1.2)
        self.pulse(ch=cfg['gen_ch'], name="mygaus", t=1.6)
        


In [ ]:
soc.xcom_0.xcom_cfg = 4

In [ ]:
soc.xcom_0.set_local_id(5)
soc.xcom_0.print_axi_regs()

In [ ]:
soc.xcom_0.run_cmd(soc.xcom_0.opcodes['XCOM_WRITE_MEM'], 2, 0x5001)

In [ ]:
print("{0:5s} {2:12s}".format('addr','\t','xcom_0_mem'))
for i in range(16):
    soc.xcom_0.axi_addr = i
    #soc.xcom_1.axi_addr = i
    print("{0:3d} {2:12d}".format(soc.xcom_0.axi_addr, '\t', soc.xcom_0.mem))
    soc.xcom_0.mem

In [ ]:
soc.xcom_0.print_axi_regs()

In [ ]:
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': TRIG_TIME,
          'ro_len': 1.9
         }

#prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
#
#iq_list = prog.acquire_decimated(soc, soft_avgs=10)
#t = prog.get_time_axis(ro_index=0)
#
#plt.plot(t, iq_list[0][:,0], label="I value")
#plt.plot(t, iq_list[0][:,1], label="Q value")
#plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
#plt.legend()
#plt.ylabel("a.u.")
#plt.xlabel("us");

In [ ]:
prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

soc.xcom_0.run_cmd(soc.xcom_0.opcodes['XCOM_QRST_SYNC'], 4, 3)
iq_list = prog.acquire_decimated(soc, soft_avgs=10, start_src='external')

t = prog.get_time_axis(ro_index=0)

plt.plot(t, iq_list[0][:,0], label="I value")
plt.plot(t, iq_list[0][:,1], label="Q value")
plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us");

In [ ]:
GEN_CHS = [0]
RO_CHS = [0]
TRIG_TIME = 120 # tProc ticks
FREQ = 500

In [ ]:
class LoopbackProgramV2(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_chs = cfg['ro_chs']
        gen_chs = cfg['gen_chs']
        for gen_ch in gen_chs:
            self.declare_gen(ch=gen_ch, nqz=1)
            
        self.add_pulse(ch=gen_chs,
                      name="mypulse",
                      ro_ch=ro_chs[0],
                      style="const",
                      freq=cfg['freq'],
                      length=cfg['pulse_len'],
                      phase=cfg['gen_phase'],
                      gain=cfg['gen_gain'],
                      )

        for ro_ch in ro_chs:
            self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        self.add_readoutconfig(ch=ro_chs,
                              name="myro",
                              freq=cfg['freq'],
                              gen_ch=gen_chs[0])

        for ro_ch in ro_chs:
            self.send_readoutconfig(ch=ro_ch,
                                   name="myro",
                                   t=0)
        if cfg['send_start']:
            self.delay(0.5)  # give processor some time to set registers
            self.trigger(pins=[0], t=0) # output a pulse on PMOD0_0, to trigger the follower
            self.delay(cfg['leader_delay']) # give follower time to catch up
                    
    def _body(self, cfg):
        for gen_ch in cfg['gen_chs']:
            self.pulse(ch=gen_ch, name="mypulse", t=0)
        self.trigger(ros=cfg['ro_chs'], t=cfg['trig_time'])
        


In [ ]:
config = \
{
    'gen_chs': GEN_CHS,
    'ro_chs': RO_CHS,
    'freq': FREQ,
    'trig_time': TRIG_TIME-0.2,
    'ro_len': 0.5,
    'pulse_len': 0.05,
    'gen_gain': 1.0,
    'gen_phase': 0.0,
    'send_start': False,
    'leader_delay': 0.05,
}

# soc, soccfg = proxies[1]
# prog = LoopbackProgramV2(soccfg, reps=1, final_delay=0.5, cfg=config)

# iq_list = prog.acquire_decimated(soc, soft_avgs=10, start_src='internal')
# t = prog.get_time_axis(ro_index=0)

# plt.plot(t, iq_list[0][:,0], label="I value")
# plt.plot(t, iq_list[0][:,1], label="Q value")
# plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
# plt.legend()
# plt.ylabel("a.u.")
# plt.xlabel("us");

In [ ]:
prog = LoopbackProgramV2(soccfg, reps=1, final_delay=0.5, cfg=config) #MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

soc.xcom_0.run_cmd(soc.xcom_0.opcodes['XCOM_QRST_SYNC'], 4, 3)
iq_list = prog.acquire_decimated(soc, soft_avgs=10, start_src='external')

t = prog.get_time_axis(ro_index=0)

plt.plot(t, iq_list[0][:,0], label="I value")
plt.plot(t, iq_list[0][:,1], label="Q value")
plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us");

In the next cell we just start the processor of board 4 using the **XCOM_QCTRL** command. 
This command will set the `start_proc` signal in board 4. 

In [ ]:
soc.xcom_0.run_cmd(soc.xcom_0.opcodes['XCOM_QCTRL'], 4, 6) #6 means start_proc